<a href="https://colab.research.google.com/github/TrieuLe0801/Zalo_competition/blob/master/ZALO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install transformers sentence-transformers
!pip install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install scikit-learn
!apt-get install default-jdk -y
!pip install pyvi
!pip install underthesea
!pip install python-Levenshtein
!pip install fuzzy-match

     |████████████████████████████████| 3.4 MB 4.0 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 61 kB 488 kB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=02ca648ac7a04151e977b55aa449bbcf73ed718821a56ea6a4ae1b353919c213
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 2.6 MB 4.3 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd
import numpy as np

Processing Json data to csv

In [ ]:
import json
import pandas as pd
import numpy as np

legal_corpus = json.load(open("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus.json"))
train_qa_data = json.load(open("/content/drive/MyDrive/Data/zac2021-ltr-data/train_question_answer.json"))
test_data = json.load(open("/content/drive/MyDrive/Data/zac2021-ltr-data/public_test_question.json"))

list_legal = []
for legal in legal_corpus:
  for item in legal['articles']:
    legal_item = {}
    legal_item.update({"law_id": legal['law_id'],
                       "article_id": item['article_id'],
                       "title":item['title'],
                       "text":item['text']
                       })
    list_legal.append(legal_item)

list_train = []
for item in train_qa_data['items']:
  for answer in item['relevant_articles']:
    train_item = {}
    train_item.update({"question_id": item['question_id'],
                       "question": item['question'],
                       "law_id": answer['law_id'],
                       "article_id": answer['article_id']
                       })
    list_train.append(train_item)

list_test = []
for item in test_data['items']:
  train_item = {}
  train_item.update({"question_id": item['question_id'],
                      "question": item['question']
                      })
  list_test.append(train_item)



legal_corpus_df = pd.DataFrame(list_legal)
train_df = pd.DataFrame(list_train)
test_df = pd.DataFrame(list_test)

list_answer_text = []
list_answer_title = []
for i in range(train_df.shape[0]):
  answer_text = legal_corpus_df[(legal_corpus_df['article_id'] == train_df['article_id'][i]) & (legal_corpus_df['law_id'] == train_df['law_id'][i])]
  list_answer_text.append(answer_text['text'].item())
  list_answer_title.append(answer_text['title'].item())

train_df['answer_title'] = list_answer_title
train_df['answer_text'] = list_answer_text

FileNotFoundError: ignored

In [ ]:
private_test_data = json.load(open("/content/drive/MyDrive/data/zac2021-ltr-data/private_test_question.json"))
list_private_test = []
for item in private_test_data['items']:
  train_item = {}
  train_item.update({"question_id": item['question_id'],
                      "question": item['question']
                      })
  list_private_test.append(train_item)
private_df = pd.DataFrame(list_private_test)

In [ ]:
private_df

,question_id,question
0,e7c03463ca8db25502489257823e8249,17 tuổi thì có được vào vũ trường?
1,165d889d535d96c0927b5bbf746fbc35,Cá nhân cần có những điều kiện nào thì sẽ được...
2,7e6e9d685fd670e2d36a0f895aafe4a6,Sổ kế toán xã phải được mở vào thời gian nào t...
3,603e64ec77bb698bb0cc03b2698bd47b,khi áp giải người bị kết án tử hình theo lệnh ...
4,7ad73ffd583870e86960182a2eee370d,Điều kiện kinh doanh vận tải hàng hóa bằng xe ...
...,...,...
486,37c999a7e2ecbb0d6543ecd96eb47e25,Hợp đồng vận chuyển bàn ghế cho sinh viên có p...
487,91b559a888165b162c076bc518ccd1af,trong thi đấu đối với môn Bóng đá tại Việt Nam...
488,bcdb794841cfcf73ab452b8b777cea2a,Mức phạt tối đa khi vi phạm hành chính của cá ...
489,2e38d64d8418a926b95cc58cf2594908,Nước mưa từ mái của nhà hàng xóm chảy sang nhà...


Load data csv

In [ ]:
import json
import pandas as pd
import numpy as np

legal_corpus_df = pd.read_csv("/content/drive/MyDrive/data/zac2021-ltr-data/legal_corpus.csv")
train_df = pd.read_csv("/content/drive/MyDrive/data/zac2021-ltr-data/train_qa_dataset.csv")
test_df = pd.read_csv("/content/drive/MyDrive/data/zac2021-ltr-data/test_qa_dataset.csv")
private_df = pd.read_csv("/content/drive/MyDrive/data/zac2021-ltr-data/private_qa_dataset.csv")

In [ ]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)/(len(a)))
    # return len(c)

In [ ]:
import Levenshtein
def get_levenshtein_sim_distance(str1, str2):
  # ratio = Levenshtein.seqratio(list(set(str1.split())), list(set(str2.split())))
  ratio = Levenshtein.ratio(str1, str2)
  return float(ratio)

In [ ]:
list_compare_pattern = [str(train_df['law_id'][i]) +" "+ str(train_df['article_id'][i]) +" "+ str(train_df['answer_title'][i]) for i in range(train_df.shape[0])]
train_df['compare_pattern'] = list_compare_pattern
train_df

,question_id,question,law_id,article_id,answer_title,answer_text,compare_pattern,norm_question,norm_title,norm_answer
0,0637bf82c8b290c7875c5bfddbf91df5,Công an xã xử phạt lỗi không mang bằng lái xe ...,47/2011/tt-bca,7,Điều 7. Nhiệm vụ của lực lượng Cảnh sát khác v...,"1. Bố trí lực lượng tham gia tuần tra, kiểm so...",47/2011/tt-bca 7 Điều 7. Nhiệm vụ của lực lượn...,công_an xã xử_phạt lỗi không mang bằng lái_xe ...,điều 7 nhiệm_vụ của lực_lượng cảnh_sát khác và...,1 bố_trí lực_lượng tham_gia tuần_tra kiểm_soát...
1,ade2b2ee4f5b869f75f0d183902382af,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,41/2020/tt-bca,11,"Điều 11. Tiến hành lấy, bảo quản mẫu",1. Chuẩn bị cho việc lấy mẫu\na) Dọn sạch khu ...,"41/2020/tt-bca 11 Điều 11. Tiến hành lấy, bảo ...",phải thực_hiện thao_tác nạp mẫu vào bình chứa ...,điều 11 tiến_hành lấy bảo_quản mẫu,1 chuẩn_bị cho việc lấy mẫu a dọn sạch khu_vực...
2,8fa164edc7a8419cd7dc9ce66d8e695a,Trình tự đánh giá chất lượng hàng năm đối với ...,159/2020/nđ-cp,13,"Điều 13. Trình tự, thủ tục đánh giá",1. Đối với người quản lý doanh nghiệp nhà nước...,"159/2020/nđ-cp 13 Điều 13. Trình tự, thủ tục đ...",trình_tự đánh_giá chất_lượng hàng năm đối_với ...,điều 13 trình_tự thủ_tục đánh_giá,1 đối_với người_quản_lý doanh_nghiệp nhà_nước ...
3,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,53/2010/qh12,60,"Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,53/2010/qh12 60 Điều 60. Giải quyết việc xin n...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,điều 60 giải_quyết việc xin nhận tử_thi hài_cố...,1 việc giải_quyết nhận tử_thi được thực_hiện n...
4,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,82/2011/nđ-cp,9,Điều 9. Chi phí mai táng,Chi phí mai táng người bị thi hành án tử hình ...,82/2011/nđ-cp 9 Điều 9. Chi phí mai táng,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,điều 9 chi_phí mai_táng,chi_phí mai_táng người bị thi_hành án tử_hình ...
...,...,...,...,...,...,...,...,...,...,...
3293,bd2f979dabdf0033a00ff488b2893e12,Nội dung kiểm tra công tác nghiệm thu công trì...,06/2021/nđ-cp,24,Điều 24. Kiểm tra công tác nghiệm thu công trì...,1. Công trình xây dựng phải được cơ quan nhà n...,06/2021/nđ-cp 24 Điều 24. Kiểm tra công tác ng...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,điều 24 kiểm_tra công_tác nghiệm_thu công_trìn...,1 công_trình xây_dựng phải được cơ_quan nhà_nư...
3294,b7347919f2bd65a2da6f9d0b1728a51c,Mức phạt đối với hành vi chuyển mục đích sử dụ...,35/2019/nđ-cp,12,Điều 12. Vi phạm quy định về chuyển mục đích s...,Hành vi chuyển mục đích sử dụng rừng sang mục ...,35/2019/nđ-cp 12 Điều 12. Vi phạm quy định về ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,điều 12 vi_phạm quy_định về chuyển mục_đích sử...,hành_vi chuyển mục_đích sử_dụng rừng sang mục_...
3295,bda416f28b8c88ce25604a26b01081cb,Trích xuất theo thi hành án hình sự được hiểu ...,41/2019/qh14,3,Điều 3. Giải thích từ ngữ,"Trong Luật này, các từ ngữ dưới đây được hiểu ...",41/2019/qh14 3 Điều 3. Giải thích từ ngữ,trích xuất theo thi_hành án hình_sự được hiểu ...,điều 3 giải_thích từ_ngữ,trong luật này các từ_ngữ dưới đây được hiểu n...
3296,6088765b2e502c7d900a3f9a2f82f2b4,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...",100/2019/nđ-cp,20,"Điều 20. Xử phạt người điều khiển xe ô tô, máy...",1. Phạt tiền từ 100.000 đồng đến 200.000 đồng ...,100/2019/nđ-cp 20 Điều 20. Xử phạt người điều ...,chạy ô_tô để dầu_nhờn rơi xuống_đường phạt bao...,điều 20 xử_phạt người điều_khiển xe ô_tô máy_k...,1 phạt tiền từ 100000 đồng đến 200000 đồng đối...


In [ ]:
# list_compare_pattern = [str(legal_corpus_df['law_id'][i]) +" "+ str(legal_corpus_df['article_id'][i]) +" "+ str(legal_corpus_df['title'][i]) for i in range(legal_corpus_df.shape[0])]
# legal_corpus_df['compare_pattern'] = list_compare_pattern
legal_corpus_df

,law_id,article_id,title,text,compare_pattern,norm_answer_title,norm_compare_pattern,norm_answer_text,norm_title
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",01/2009/tt-bnn 1 Điều 1. Phạm vi áp dụng,điều 1 phạm_vi áp_dụng,012009ttbnn 1 điều 1 phạm_vi áp_dụng,thông_tư này hướng_dẫn tuần_tra canh_gác bảo_v...,điều 1 phạm_vi áp_dụng
1,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",01/2009/tt-bnn 2 Điều 2. Tổ chức lực lượng,điều 2 tổ_chức lực_lượng,012009ttbnn 2 điều 2 tổ_chức lực_lượng,1 hàng năm trước mùa mưa_lũ uỷ_ban_nhân_dân cấ...,điều 2 tổ_chức lực_lượng
2,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,"1. Là người khoẻ mạnh, tháo vát, đủ khả năng đ...",01/2009/tt-bnn 3 Điều 3. Tiêu chuẩn của các th...,điều 3 tiêu_chuẩn của các thành_viên thuộc lực...,012009ttbnn 3 điều 3 tiêu_chuẩn của các thành_...,1 là người khoẻ_mạnh tháo_vát đủ khả_năng đảm_...,điều 3 tiêu_chuẩn của các thành_viên thuộc lực...
3,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",1. Chấp hành sự phân công của Ban chỉ huy phòn...,01/2009/tt-bnn 4 Điều 4. Nhiệm vụ của lực lượn...,điều 4 nhiệm_vụ của lực_lượng tuần_tra canh_gá...,012009ttbnn 4 điều 4 nhiệm_vụ của lực_lượng tu...,1 chấp_hành sự phân_công của ban chỉ_huy phòng...,điều 4 nhiệm_vụ của lực_lượng tuần_tra canh_gá...
4,01/2009/tt-bnn,5,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...","Phù hiệu của lực lượng tuần tra, canh gác đê l...",01/2009/tt-bnn 5 Điều 5. Phù hiệu của lực lượn...,điều 5 phù_hiệu của lực_lượng tuần_tra canh_gá...,012009ttbnn 5 điều 5 phù_hiệu của lực_lượng tu...,phù_hiệu của lực_lượng tuần_tra canh_gác đê là...,điều 5 phù_hiệu của lực_lượng tuần_tra canh_gá...
...,...,...,...,...,...,...,...,...,...
61420,99/2020/nđ-cp,60,Điều 60. Thẩm quyền xử phạt của Hải quan,"1. Chi cục trưởng Chi cục Hải quan, Chi cục tr...",99/2020/nđ-cp 60 Điều 60. Thẩm quyền xử phạt c...,điều 60 thẩm_quyền xử_phạt của hải_quan,992020nđcp 60 điều 60 thẩm_quyền xử_phạt của h...,1 chi_cục trưởng chi_cục hải_quan chi_cục trưở...,điều 60 thẩm_quyền xử_phạt của hải_quan
61421,99/2020/nđ-cp,61,Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...,1. Kiểm soát viên thị trường đang thi hành côn...,99/2020/nđ-cp 61 Điều 61. Thẩm quyền xử phạt c...,điều 61 thẩm_quyền xử_phạt của quản_lý thị_trường,992020nđcp 61 điều 61 thẩm_quyền xử_phạt của q...,1 kiểm_soát_viên thị_trường đang thi_hành công...,điều 61 thẩm_quyền xử_phạt của quản_lý thị_trường
61422,99/2020/nđ-cp,62,Điều 62. Thẩm quyền xử phạt của Thanh tra,"1. Thanh tra viên, người được giao thực hiện n...",99/2020/nđ-cp 62 Điều 62. Thẩm quyền xử phạt c...,điều 62 thẩm_quyền xử_phạt của thanh_tra,992020nđcp 62 điều 62 thẩm_quyền xử_phạt của t...,1 thanh_tra_viên người được giao thực_hiện nhi...,điều 62 thẩm_quyền xử_phạt của thanh_tra
61423,99/2020/nđ-cp,63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,99/2020/nđ-cp 63 Điều 63. Phân định thẩm quyền...,điều 63 phân_định thẩm_quyền xử_phạt của chủ_t...,992020nđcp 63 điều 63 phân_định thẩm_quyền xử_...,1 phân_định thẩm_quyền xử_phạt của chủ_tịch uỷ...,điều 63 phân_định thẩm_quyền xử_phạt của chủ_t...


In [ ]:
legal_corpus_df.to_csv("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus.csv",
                       index=False,
                       encoding='utf_8')
train_df.to_csv("/content/drive/MyDrive/Data/zac2021-ltr-data/train_qa_dataset.csv",
                index=False,
                encoding='utf_8')
test_df.to_csv("/content/drive/MyDrive/Data/zac2021-ltr-data/test_qa_dataset.csv",
                index=False,
                encoding='utf_8')

In [ ]:
private_df.to_csv("/content/drive/MyDrive/data/zac2021-ltr-data/private_qa_dataset.csv",
                index=False,
                encoding='utf_8')

Normalize function

In [ ]:
from vncorenlp import VnCoreNLP
import re
import string
from pyvi.ViTokenizer import tokenize
import requests
from typing import List
from underthesea import word_tokenize
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
def normlization(text: str,
                 remove_punc: bool = False,
                 remove_stopword: bool = False) -> str:
    text = str(text).lower()
    text = text.replace("nđ-","nd-") if "nđ-" in text else text
    # text = re.sub('\n\n', " ", text)
    text = re.sub('\n', " ", text)
    text = re.sub('\xa0', "", text)
    text = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", text)
    text = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", text)
    text = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", text)

    # remove punctuation
    if remove_punc:
        text = text.translate(str.maketrans('', '', string.punctuation))
    # word segmentation
    try:
      tokens = rdrsegmenter.tokenize(text)
    except:
        # Whoops it wasn't a 200
      tokens = tokenize(text)
    if isinstance(tokens, List):
      tokens = [item for sublist in tokens for item in sublist]
    else:
      tokens = tokens.split(" ")

    # remove stopwords
    if remove_stopword:
        stopwords = load_data_npy(Config.STOPWORDS)
        tokens = [t for t in tokens if t not in stopwords]
    if remove_punc:
        exclude = set(string.punctuation)
        normal_text = ' '.join(ch for ch in tokens if ch not in exclude and ch != "“" and ch != "”")
    else:
        normal_text = ' '.join(ch for ch in tokens)
    return normal_text

In [ ]:
list_questions = [normlization(t, remove_punc=True) for t in train_df['question']]
list_title = [normlization(t, remove_punc=True) for t in train_df['answer_title']]
list_answer = [normlization(t, remove_punc=True) for t in train_df['answer_text']]
list_compare = [normlization(t, remove_punc=True) for t in train_df['compare_pattern']]

In [ ]:
print(list_compare[0])

In [ ]:
list_answer_title_corpus = [normlization(t, remove_punc=True) for t in list(legal_corpus_df['title'])]

In [ ]:
list_compare_pattern_corpus = [normlization(t, remove_punc=True) for t in list(legal_corpus_df['compare_pattern'])]

In [ ]:
list_answer_text_corpus = [normlization(t, remove_punc=True) for t in legal_corpus_df['text']]
# for i,text in enumerate(list(legal_corpus_df['text'])):
#   norm_text = normlization(text, remove_punc=True)
#   print(i)
# print(type(legal_corpus_df['article_id'][309]))

In [ ]:
# legal_corpus_df['norm_answer_title'] = list_answer_title_corpus
# legal_corpus_df['norm_compare_pattern'] = list_compare_pattern_corpus
# legal_corpus_df['norm_answer_text'] = list_answer_text_corpus
legal_corpus_df

,law_id,article_id,title,text,compare_pattern,norm_answer_title,norm_compare_pattern,norm_answer_text,norm_title
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",01/2009/tt-bnn 1 Điều 1. Phạm vi áp dụng,điều 1 phạm_vi áp_dụng,012009ttbnn 1 điều 1 phạm_vi áp_dụng,thông_tư này hướng_dẫn tuần_tra canh_gác bảo_v...,điều 1 phạm_vi áp_dụng
1,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",01/2009/tt-bnn 2 Điều 2. Tổ chức lực lượng,điều 2 tổ_chức lực_lượng,012009ttbnn 2 điều 2 tổ_chức lực_lượng,1 hàng năm trước mùa mưa_lũ uỷ_ban_nhân_dân cấ...,điều 2 tổ_chức lực_lượng
2,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,"1. Là người khoẻ mạnh, tháo vát, đủ khả năng đ...",01/2009/tt-bnn 3 Điều 3. Tiêu chuẩn của các th...,điều 3 tiêu_chuẩn của các thành_viên thuộc lực...,012009ttbnn 3 điều 3 tiêu_chuẩn của các thành_...,1 là người khoẻ_mạnh tháo_vát đủ khả_năng đảm_...,điều 3 tiêu_chuẩn của các thành_viên thuộc lực...
3,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",1. Chấp hành sự phân công của Ban chỉ huy phòn...,01/2009/tt-bnn 4 Điều 4. Nhiệm vụ của lực lượn...,điều 4 nhiệm_vụ của lực_lượng tuần_tra canh_gá...,012009ttbnn 4 điều 4 nhiệm_vụ của lực_lượng tu...,1 chấp_hành sự phân_công của ban chỉ_huy phòng...,điều 4 nhiệm_vụ của lực_lượng tuần_tra canh_gá...
4,01/2009/tt-bnn,5,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...","Phù hiệu của lực lượng tuần tra, canh gác đê l...",01/2009/tt-bnn 5 Điều 5. Phù hiệu của lực lượn...,điều 5 phù_hiệu của lực_lượng tuần_tra canh_gá...,012009ttbnn 5 điều 5 phù_hiệu của lực_lượng tu...,phù_hiệu của lực_lượng tuần_tra canh_gác đê là...,điều 5 phù_hiệu của lực_lượng tuần_tra canh_gá...
...,...,...,...,...,...,...,...,...,...
61420,99/2020/nđ-cp,60,Điều 60. Thẩm quyền xử phạt của Hải quan,"1. Chi cục trưởng Chi cục Hải quan, Chi cục tr...",99/2020/nđ-cp 60 Điều 60. Thẩm quyền xử phạt c...,điều 60 thẩm_quyền xử_phạt của hải_quan,992020ndcp 60 điều 60 thẩm_quyền xử_phạt của h...,1 chi_cục trưởng chi_cục hải_quan chi_cục trưở...,điều 60 thẩm_quyền xử_phạt của hải_quan
61421,99/2020/nđ-cp,61,Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...,1. Kiểm soát viên thị trường đang thi hành côn...,99/2020/nđ-cp 61 Điều 61. Thẩm quyền xử phạt c...,điều 61 thẩm_quyền xử_phạt của quản_lý thị_trường,992020ndcp 61 điều 61 thẩm_quyền xử_phạt của q...,1 kiểm_soát_viên thị_trường đang thi_hành công...,điều 61 thẩm_quyền xử_phạt của quản_lý thị_trường
61422,99/2020/nđ-cp,62,Điều 62. Thẩm quyền xử phạt của Thanh tra,"1. Thanh tra viên, người được giao thực hiện n...",99/2020/nđ-cp 62 Điều 62. Thẩm quyền xử phạt c...,điều 62 thẩm_quyền xử_phạt của thanh_tra,992020ndcp 62 điều 62 thẩm_quyền xử_phạt của t...,1 thanh_tra_viên người được giao thực_hiện nhi...,điều 62 thẩm_quyền xử_phạt của thanh_tra
61423,99/2020/nđ-cp,63,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,1. Phân định thẩm quyền xử phạt của Chủ tịch Ủ...,99/2020/nđ-cp 63 Điều 63. Phân định thẩm quyền...,điều 63 phân_định thẩm_quyền xử_phạt của chủ_t...,992020ndcp 63 điều 63 phân_định thẩm_quyền xử_...,1 phân_định thẩm_quyền xử_phạt của chủ_tịch uỷ...,điều 63 phân_định thẩm_quyền xử_phạt của chủ_t...


In [ ]:
# train_df['norm_question'] = list_questions
# train_df['norm_title'] = list_title
# train_df['norm_answer'] = list_answer
train_df
# train_df['norm_title'] = list_title

,question_id,question,law_id,article_id,answer_title,answer_text,compare_pattern,norm_question,norm_title,norm_answer
0,0637bf82c8b290c7875c5bfddbf91df5,Công an xã xử phạt lỗi không mang bằng lái xe ...,47/2011/tt-bca,7,Điều 7. Nhiệm vụ của lực lượng Cảnh sát khác v...,"1. Bố trí lực lượng tham gia tuần tra, kiểm so...",47/2011/tt-bca 7 Điều 7. Nhiệm vụ của lực lượn...,công_an xã xử_phạt lỗi không mang bằng lái_xe ...,điều 7 nhiệm_vụ của lực_lượng cảnh_sát khác và...,1 bố_trí lực_lượng tham_gia tuần_tra kiểm_soát...
1,ade2b2ee4f5b869f75f0d183902382af,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,41/2020/tt-bca,11,"Điều 11. Tiến hành lấy, bảo quản mẫu",1. Chuẩn bị cho việc lấy mẫu\na) Dọn sạch khu ...,"41/2020/tt-bca 11 Điều 11. Tiến hành lấy, bảo ...",phải thực_hiện thao_tác nạp mẫu vào bình chứa ...,điều 11 tiến_hành lấy bảo_quản mẫu,1 chuẩn_bị cho việc lấy mẫu a dọn sạch khu_vực...
2,8fa164edc7a8419cd7dc9ce66d8e695a,Trình tự đánh giá chất lượng hàng năm đối với ...,159/2020/nđ-cp,13,"Điều 13. Trình tự, thủ tục đánh giá",1. Đối với người quản lý doanh nghiệp nhà nước...,"159/2020/nđ-cp 13 Điều 13. Trình tự, thủ tục đ...",trình_tự đánh_giá chất_lượng hàng năm đối_với ...,điều 13 trình_tự thủ_tục đánh_giá,1 đối_với người_quản_lý doanh_nghiệp nhà_nước ...
3,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,53/2010/qh12,60,"Điều 60. Giải quyết việc xin nhận tử thi, hài ...",1. Việc giải quyết nhận tử thi được thực hiện ...,53/2010/qh12 60 Điều 60. Giải quyết việc xin n...,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,điều 60 giải_quyết việc xin nhận tử_thi hài_cố...,1 việc giải_quyết nhận tử_thi được thực_hiện n...
4,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,82/2011/nđ-cp,9,Điều 9. Chi phí mai táng,Chi phí mai táng người bị thi hành án tử hình ...,82/2011/nđ-cp 9 Điều 9. Chi phí mai táng,tử_tù chết sau khi tiêm thuốc_độc mà không có ...,điều 9 chi_phí mai_táng,chi_phí mai_táng người bị thi_hành án tử_hình ...
...,...,...,...,...,...,...,...,...,...,...
3293,bd2f979dabdf0033a00ff488b2893e12,Nội dung kiểm tra công tác nghiệm thu công trì...,06/2021/nđ-cp,24,Điều 24. Kiểm tra công tác nghiệm thu công trì...,1. Công trình xây dựng phải được cơ quan nhà n...,06/2021/nđ-cp 24 Điều 24. Kiểm tra công tác ng...,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,điều 24 kiểm_tra công_tác nghiệm_thu công_trìn...,1 công_trình xây_dựng phải được cơ_quan nhà_nư...
3294,b7347919f2bd65a2da6f9d0b1728a51c,Mức phạt đối với hành vi chuyển mục đích sử dụ...,35/2019/nđ-cp,12,Điều 12. Vi phạm quy định về chuyển mục đích s...,Hành vi chuyển mục đích sử dụng rừng sang mục ...,35/2019/nđ-cp 12 Điều 12. Vi phạm quy định về ...,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,điều 12 vi_phạm quy_định về chuyển mục_đích sử...,hành_vi chuyển mục_đích sử_dụng rừng sang mục_...
3295,bda416f28b8c88ce25604a26b01081cb,Trích xuất theo thi hành án hình sự được hiểu ...,41/2019/qh14,3,Điều 3. Giải thích từ ngữ,"Trong Luật này, các từ ngữ dưới đây được hiểu ...",41/2019/qh14 3 Điều 3. Giải thích từ ngữ,trích xuất theo thi_hành án hình_sự được hiểu ...,điều 3 giải_thích từ_ngữ,trong luật này các từ_ngữ dưới đây được hiểu n...
3296,6088765b2e502c7d900a3f9a2f82f2b4,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...",100/2019/nđ-cp,20,"Điều 20. Xử phạt người điều khiển xe ô tô, máy...",1. Phạt tiền từ 100.000 đồng đến 200.000 đồng ...,100/2019/nđ-cp 20 Điều 20. Xử phạt người điều ...,chạy ô_tô để dầu_nhờn rơi xuống_đường phạt bao...,điều 20 xử_phạt người điều_khiển xe ô_tô máy_k...,1 phạt tiền từ 100000 đồng đến 200000 đồng đối...


In [ ]:
# test_df
# legal_corpus_df['norm_compare_pattern'] = list_compare_pattern_corpus
# legal_corpus_df['norm_title'] = list_answer_title_corpus

legal_corpus_df.to_csv("/content/drive/MyDrive/data/zac2021-ltr-data/legal_corpus.csv",
                       index=False,
                       encoding='utf8')

Sentenc Transformer 

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, util, CrossEncoder
from sentence_transformers.losses import TripletLoss
from sentence_transformers import losses
from sentence_transformers.readers import LabelSentenceReader, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import models
import math
import torch
from fuzzy_match import algorithims

In [ ]:
word_embedding_model = models.Transformer('/content/drive/MyDrive/data/model_qa/simcse_phobert_2021-12-27', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
sbert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cuda:0')

In [ ]:
# sbert_model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
train_examples = []
for i in range(train_df.shape[0]):
  input = InputExample(texts=[train_df['norm_question'][i], train_df['norm_title'][i]])
  train_examples.append(input)
train_dataset = SentencesDataset(train_examples, sbert_model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=sbert_model)
warmup_steps = math.ceil(len(train_dataloader) * 1 * 0.1)

In [ ]:
import os
import datetime 
save_model_dir = os.path.join("/content/drive/MyDrive/data/model_qa",'simcse_phobert_{}_1'.format(datetime.date.today()))

if os.path.isdir(save_model_dir) == False:
    os.mkdir(save_model_dir)
sbert_model.fit(train_objectives=[(train_dataloader, train_loss)],
          # evaluator=dev_evaluator,
          epochs=5,
          # evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          output_path=save_model_dir,
          optimizer_params={'lr': 2e-5},
          use_amp=True         #Set to True, if your GPU supports FP16 cores
          )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/104 [00:00<?, ?it/s]

Iteration:   0%|          | 0/104 [00:00<?, ?it/s]

Iteration:   0%|          | 0/104 [00:00<?, ?it/s]

Iteration:   0%|          | 0/104 [00:00<?, ?it/s]

Iteration:   0%|          | 0/104 [00:00<?, ?it/s]

In [ ]:
# print(save_model_dir)
model = SentenceTransformer(save_model_dir)
# model = SentenceTransformer("/content/drive/MyDrive/data/model_qa/simcse_phobert_2021-12-08_5")

In [ ]:
text = private_df['question'][25]
print(text)
query_encode = model.encode(normlization(text,True))
# answer_title_encode= model.encode(list(legal_corpus_df['title']))

Nội dung chính sách hỗ trợ tưới tiên tiến, tiết kiệm nước gồm những gì?


In [ ]:
answer_title_encode= model.encode(list(legal_corpus_df['norm_answer_title']))
print(answer_title_encode.shape)
np.save("/content/drive/MyDrive/data/zac2021-ltr-data/legal_corpus_title.npy",np.array(answer_title_encode))
# answer_title_encode = np.load("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus_title.npy", allow_pickle=True)

(61425, 768)


In [ ]:
compare_encode= model.encode(list(legal_corpus_df['norm_compare_pattern']))
print(compare_encode.shape)
np.save("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus_compare_pattern.npy",np.array(compare_encode))
# compare_encode = np.load("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus_compare_pattern.npy", allow_pickle=True).tolist()

KeyboardInterrupt: ignored

In [ ]:
# list_answer_text_corpus = [normlization(t, remove_punc=True) for t in list(legal_corpus_df['text'])]
# answer_text_encode= model.encode(list(legal_corpus_df['norm_answer_text']))
# print(answer_text_encode.shape)
# np.save("/content/drive/MyDrive/Data/zac2021-ltr-data/legal_corpus_text.npy",np.array(answer_text_encode))

In [ ]:
top_results = util.semantic_search(query_encode, answer_title_encode, top_k=60)[0]
# top_results = torch.topk(cos_scores, k=5)?

In [ ]:
for result in top_results:
  print(legal_corpus_df['text'][result['corpus_id']])
  print(legal_corpus_df['law_id'][result['corpus_id']])
  print(legal_corpus_df['article_id'][result['corpus_id']])
  result['cross-score'] = get_jaccard_sim(str(normlization(text,True)), 
                                                            str(legal_corpus_df['norm_answer_text'][result['corpus_id']]))
  result['convert'] = round(result['score']*2 + result['cross-score'],3)
  print("(Score: {:.4f})".format(result['score']))
  print("(Score cross: {:.4f})".format(result['cross-score']))
  print("(Score convert: {:.4f})".format(result['convert']))

1. Nội dung chính sách hỗ trợ:
a) Hỗ trợ tối đa 50% chi phí vật liệu, máy thi công và thiết bị để đầu tư xây dựng hệ thống tưới tiên tiến, tiết kiệm nước cho cây trồng cạn, mức hỗ trợ không quá 40 triệu đồng/ha;
b) Hỗ trợ tối đa 50% chi phí để san phẳng đồng ruộng, mức hỗ trợ không quá 10 triệu đồng/ha.
2. Điều kiện hỗ trợ:
Tổ chức, cá nhân phải đáp ứng các điều kiện sau đây:
a) Đối với cá nhân: Quy mô khu tưới phải đạt từ 0,3 ha trở lên, riêng khu vực miền núi từ 0,1 ha trở lên; việc hỗ trợ cho cá nhân được thông qua tổ chức thủy lợi cơ sở;
b) Đối với tổ chức thủy lợi cơ sở: Quy mô khu tưới phải đạt từ 02 ha trở lên, riêng khu vực miền núi từ 01 ha trở lên và phải có hợp đồng liên kết với hộ gia đình hoặc cá nhân trực tiếp sản xuất;
c) Hệ thống tưới tiên tiến, tiết kiệm nước phục vụ các loại cây trồng là cây chủ lực của quốc gia, địa phương, có lợi thế, phù hợp với nhu cầu thị trường và thích ứng với biến đổi khí hậu từng vùng, miền.
77/2018/nđ-cp
5
(Score: 0.8169)
(Score cross: 0.700

In [ ]:
test_quest = list(private_df['question'])
norm_quest = [normlization(t, True) for t in test_quest]
question_encode = model.encode(norm_quest)
print(question_encode.shape)

(491, 768)


In [ ]:
list_top_1 = util.semantic_search(question_encode, answer_title_encode, top_k=32)
print(len(list_top_1))

491


In [ ]:
# list_law_id = []
# list_article_id = []
# list_text = []
import Levenshtein as lev
list_answers = []
for i,result in enumerate(list_top_1):
  # cross_inp = [[str(norm_quest[i]), str(legal_corpus_df['norm_answer_text'][hit['corpus_id']])] for hit in result]
  # cross_scores = cross_encoder.predict(cross_inp)
  for idx in range(len(result)):
    result[idx]['cross-score'] = get_jaccard_sim(str(norm_quest[i]), str(legal_corpus_df['norm_answer_text'][result[idx]['corpus_id']]))
    result[idx]['convert'] = round(result[idx]['score']*2 + result[idx]['cross-score'],3)
  new_list = sorted(result,key=lambda d: d['convert'], reverse=True)
  list_results = []
  list_results.append({"law_id":legal_corpus_df['law_id'][new_list[0]['corpus_id']],
                   "article_id":str(legal_corpus_df['article_id'][new_list[0]['corpus_id']])})
  if new_list[1]['score'] >= 0.8:
    list_results.append({"law_id":legal_corpus_df['law_id'][new_list[1]['corpus_id']],
                   "article_id":str(legal_corpus_df['article_id'][new_list[1]['corpus_id']])})
  # list_results = [{"law_id":legal_corpus_df['law_id'][r['corpus_id']],
  #                  "article_id":str(legal_corpus_df['article_id'][r['corpus_id']])} for r in new_list[0]]

  list_answers.append(list_results)

print(len(list_answers))
private_df['answer'] = list_answers

491


In [ ]:
private_df

,question_id,question,answer
0,e7c03463ca8db25502489257823e8249,17 tuổi thì có được vào vũ trường?,"[{'law_id': '17/2016/tt-bqp', 'article_id': '1..."
1,165d889d535d96c0927b5bbf746fbc35,Cá nhân cần có những điều kiện nào thì sẽ được...,"[{'law_id': '16/2017/qh14', 'article_id': '81'}]"
2,7e6e9d685fd670e2d36a0f895aafe4a6,Sổ kế toán xã phải được mở vào thời gian nào t...,"[{'law_id': '70/2019/tt-btc', 'article_id': '5'}]"
3,603e64ec77bb698bb0cc03b2698bd47b,khi áp giải người bị kết án tử hình theo lệnh ...,"[{'law_id': '39/2012/tt-bca', 'article_id': '1..."
4,7ad73ffd583870e86960182a2eee370d,Điều kiện kinh doanh vận tải hàng hóa bằng xe ...,"[{'law_id': '10/2020/nđ-cp', 'article_id': '9'}]"
...,...,...,...
486,37c999a7e2ecbb0d6543ecd96eb47e25,Hợp đồng vận chuyển bàn ghế cho sinh viên có p...,"[{'law_id': '10/2020/nđ-cp', 'article_id': '15'}]"
487,91b559a888165b162c076bc518ccd1af,trong thi đấu đối với môn Bóng đá tại Việt Nam...,"[{'law_id': '02/2014/tt-bct', 'article_id': '2..."
488,bcdb794841cfcf73ab452b8b777cea2a,Mức phạt tối đa khi vi phạm hành chính của cá ...,"[{'law_id': '96/2020/nđ-cp', 'article_id': '4'}]"
489,2e38d64d8418a926b95cc58cf2594908,Nước mưa từ mái của nhà hàng xóm chảy sang nhà...,"[{'law_id': '91/2015/qh13', 'article_id': '250'}]"


In [ ]:
i = 50
print(private_df['question'][i])
print(private_df['answer'][i])

khi nào thực hiện kiểm tra, xác minh xuất xứ hàng hóa tại cơ sở sản xuất
[{'law_id': '39/2018/tt-bct', 'article_id': '11'}]


In [ ]:
list_result = []
for i in range(private_df.shape[0]):
  data = {}
  # list_answer.append(answer)
  data.update({
      "question_id":private_df['question_id'][i],
      "relevant_articles":private_df['answer'][i]
  })
  list_result.append(data)

In [ ]:
import json
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()
with open('/content/drive/MyDrive/data/zac2021-ltr-data/trieulv_public_test_answer.json', 'w') as fout:
    json.dump(list_result, fout)

In [ ]:
xdatetime.date.today()

datetime.date(2021, 11, 29)

In [ ]:
train_df.loc[train_df['question_id'] == "d5b4e85c19b9fd90a22fdc4733b19141"]

,question_id,question,law_id,article_id,answer_title,answer_text,compare_pattern,norm_question,norm_title,norm_answer
375,d5b4e85c19b9fd90a22fdc4733b19141,Giáo viên dạy thực hành chạy sai tuyến đường t...,100/2019/nđ-cp,37,Điều 37. Xử phạt các hành vi vi phạm quy định ...,1. Phạt tiền từ 600.000 đồng đến 800.000 đồng ...,100/2019/nđ-cp 37 Điều 37. Xử phạt các hành vi...,giáo_viên dạy thực_hành chạy sai tuyến đường t...,điều 37 xử_phạt các hành_vi vi_phạm quy_định v...,1 phạt tiền từ 600000 đồng đến 800000 đồng đối...
